# Imports and setting up viz

NB : conda lam1env (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *
from tools_native import *
from tools_LIAISE import *
from tools_mesoNH import *

# sys.path.append('../../python_tools/aborella/PLOTS/')
# from datasets import *
sys.path.append('../../python_tools/aborella/UTIL/')
import xr_utils

In [ ]:
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [10., 8.]

# Load files

## LIAISE site obs

In [ ]:
#load all July obs in one ds
cen_mto_filename='../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2_2021-07*_V3.nc'
cen_mto = format_Cendrosa_obs(cen_mto_filename)
cen_mto

In [ ]:
start_time = '2021-07-14T01:00:00'
end_time = '2021-07-30T23:00:00'
restrict_cen_mto = filter_xarray_by_timestamps(cen_mto, start_time, end_time)

In [ ]:
filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210721_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
els_mto_2107 = format_ElsPlans_obs(ds, start_day='2021-07-21')

filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210722_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
els_mto_2207 = format_ElsPlans_obs(ds, start_day='2021-07-22')

In [ ]:
july_days=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
for day in july_days:
    filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_202107{}_V4.0.dat'.format(day)
    ds = dat_to_xarray(filename, 152, 153)
    day_els_mto = format_ElsPlans_obs(ds, start_day='2021-07-{}'.format(day))
    if day=='01':
        els_mto=day_els_mto
    else:
        els_mto=xr.concat([els_mto,day_els_mto],dim='time')
els_mto

In [ ]:
restrict_els_mto = filter_xarray_by_timestamps(els_mto, start_time, end_time)

## MesoNH sim

In [ ]:
mesoNH_dir = '../../../mesoNH_simulations'

In [ ]:
def format_MesoNH_outputs_basic(filename, ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name

    rename_dict={
        'time':'start_time',
        'longitude':'lon',
        'latitude':'lat'
    }
    ds = ds.rename(rename_dict)

    # Add time coordinate
    # Assuming 'time' coordinate is the starting time.
    base_time = pd.to_datetime(ds["start_time"].isel(start_time=0).item())
    print(f"Base time for the dataset: {base_time}")
    # Create a timedelta for each hour
    hourly_offsets = pd.to_timedelta(ds["record"].values, unit='h')
    new_time_vals = base_time + hourly_offsets
    # Assign new_time as a new coordinate, linked to the 'record' dimension
    ds = ds.assign_coords(time=("record", new_time_vals))
    ds = ds.swap_dims({"record": "time"})
    #drop start_time and record
    ds = ds.drop_vars(['start_time'])
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)

    if 'LE_ISBA' in ds:
        ds = ds.rename({'LE_ISBA': 'flat'})
        ds['flat'].attrs['long_name'] = 'Latent Heat Flux on natural areas'
        ds['flat'].attrs['units'] = 'W m⁻²'
    if 'H_ISBA' in ds:
        ds = ds.rename({'H_ISBA': 'sens'})
        ds['sens'].attrs['long_name'] = 'Sensible Heat Flux on natural areas'
        ds['sens'].attrs['units'] = 'W m⁻²'
    if 'HU2M_ISBA' in ds:
        ds['rh2m'] = ds['HU2M_ISBA'] * 100.0 
        ds['rh2m'].attrs['long_name'] = 'Relative Humidity at 2m'
        ds['rh2m'].attrs['units'] = '%'
    if 'T2M_ISBA' in ds:
        ds = ds.rename({'T2M_ISBA': 't2m'})
        ds['t2m'] = ds['t2m'] - 273.15
        ds['t2m'].attrs['units'] = '°C'
        ds['t2m'].attrs['long_name'] = 'Temperature at 2m'
    if 'SWD' in ds:
        #remove dimension 'start_time' from SWD
        ds['SWD'] = ds['SWD'].squeeze('start_time', drop=True)
        ds = ds.rename({'SWD': 'SWdnSFC'})
        ds['SWdnSFC'].attrs['long_name'] = 'Downward Shortwave Radiation at Surface'
        ds['SWdnSFC'].attrs['units'] = 'W m⁻²'
    if 'LWD' in ds:
        #remove dimension 'start_time' from LWD
        ds['LWD'] = ds['LWD'].squeeze('start_time', drop=True)
    if 'LWU' in ds:
        #remove dimension 'start_time' from LWU
        ds['LWU'] = ds['LWU'].squeeze('start_time', drop=True)
    if 'SWU' in ds:
        #remove dimension 'start_time' from SWU
        ds['SWU'] = ds['SWU'].squeeze('start_time', drop=True)
    if 'HBLTOP' in ds:
        #remove dimension 'start_time' from HBLTOP
        ds['HBLTOP'] = ds['HBLTOP'].squeeze('start_time', drop=True)

    # remove ds history attribute
    if 'history' in ds.attrs:
        del ds.attrs['history']

    return(ds)

In [ ]:
filename='{}/LIAIS.1*.nc'.format(mesoNH_dir)
# filename='{}/LIAIS.1.UT10.nc'.format(mesoNH_dir)
mesoNH = format_MesoNH_outputs_basic(filename)
mesoNH

In [ ]:
load_4D=True
def format_MesoNH_outputs_4D(filename, ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name


    rename_dict={
        'time':'start_time',
        'longitude':'lon',
        'latitude':'lat'
    }
    ds = ds.rename(rename_dict)

    # Add time coordinate
    # Assuming 'time' coordinate is the starting time.
    base_time = pd.to_datetime(ds["start_time"].isel(start_time=0).item())
    print(f"Base time for the dataset: {base_time}")
    #Create a timedelta for each hour
    hourly_offsets = pd.to_timedelta(ds["record"].values, unit='h')
    new_time_vals = base_time + hourly_offsets
    # Assign new_time as a new coordinate, linked to the 'record' dimension
    ds = ds.assign_coords(time=("record", new_time_vals))
    ds = ds.swap_dims({"record": "time"})
    #drop start_time
    # ds = ds.drop_vars(['start_time'])
    # ds = ds.drop_dims(['start_time'])
    ds = ds.squeeze('start_time', drop=True)

    # ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)

    #remove ds history attribute
    if 'history' in ds.attrs:
        del ds.attrs['history']
    return ds

# if load_4D:
# filename='{}/4D/LIAIS.1.SEG16.012subset_abl_6var.nc'.format(mesoNH_dir)
filename='{}/4D/LIAIS.1.subset_abl_8vars.nc'.format(mesoNH_dir)

mesoNH_4D = format_MesoNH_outputs_4D(filename)
mesoNH_4D

In [ ]:
mesoNH

In [ ]:
#extract pressure at surface from mesoNH_4D and add it to mesoNH
mesoNH = extract_pressure_surface(mesoNH_4D, mesoNH)
#add specific humidity at 2m to mesoNH
mesoNH = add_specific_humidity_2m(mesoNH, t2m_in_kelvin=False, psol_Pa=True)
mesoNH

## Subdataset for ICOLMDZ grid cells

In [ ]:
mesoNH_elsplans_cell= subset_dataset_by_polygon(mesoNH, elsplans_cell, 
                                            name='mesoNH_mean',
                                            plot_color='orange')
mesoNH_cendrosa_cell= subset_dataset_by_polygon(mesoNH, cendrosa_cell, 
                                            name='mesoNH_mean',
                                            plot_color='orange')

In [ ]:
#select subdataset for precisely each site
cendrosa_exact = select_dataset_lon_lat(mesoNH, 
                                        lon=Cendrosa_Longitude, lat=Cendrosa_Latitude, 
                                        name='mesoNH_exact',
                                        plot_color='purple')

In [ ]:
elsplans_exact = select_dataset_lon_lat(mesoNH, 
                                        lon=ElsPlans_Longitude, lat=ElsPlans_Latitude, 
                                        name='mesoNH_exact', 
                                        plot_color='purple')

## Load LMDZ files

In [ ]:
nat_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/halfhour_native'

boost_irrig_dir= '../../../JZ_simu_outputs/LAM/LAM_1500_60/boost_irrig'

In [ ]:
noirr=True
irr100=True
routing_restart_full_beta1=True


In [ ]:
#load LMDZ
if noirr:
    lmdz_noirr_file = '{}/sim267_20210701_20210731_HF_histhf.nc'.format(nat_dir)
    lmdz_noirr = format_lmdz_HF(lmdz_noirr_file, 'red', 'noirr')
    lmdz_noirr = lmdz_noirr.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_noirr, els_lmdz_noirr  , sop_cen_lmdz_noirr,   sop_els_lmdz_noirr = select_liaise_sites_sop(lmdz_noirr, 'no_irr')


In [ ]:
if irr100:
    dir=boost_irrig_dir
    sim_name='sim275'
    color='blue'
    label='irr'

    lmdz_irr100_file = '{}/{}_20210701_20210731_HF_histhf.nc'.format(dir, sim_name)
    lmdz_irr100 = format_lmdz_HF(lmdz_irr100_file, color, label)
    lmdz_irr100 = lmdz_irr100.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_irr100, els_lmdz_irr100  , sop_cen_lmdz_irr100,   sop_els_lmdz_irr100 = select_liaise_sites_sop(lmdz_irr100, label)


In [ ]:
if routing_restart_full_beta1 :
    dir=boost_irrig_dir
    sim_name='sim293'
    color='green'
    label='irr_boost'

    lmdz_routingfull_beta1_file = '{}/{}_20210701_20210731_HF_histhf.nc'.format(dir, sim_name)
    lmdz_routingfull_beta1 = format_lmdz_HF(lmdz_routingfull_beta1_file, color, label)
    lmdz_routingfull_beta1 = lmdz_routingfull_beta1.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_routingfull_beta1, els_lmdz_routingfull_beta1, sop_cen_lmdz_routingfull_beta1, sop_els_lmdz_routingfull_beta1 = select_liaise_sites_sop(lmdz_routingfull_beta1, label)

# Tests

## Maps

In [ ]:
ds=mesoNH
var='t2m'

mean_ds = ds[var].mean(dim='time')
# mean_ds = mean_ds[0,:,:]  # Select the specific level
nice_map_mesoNH(mean_ds, poly=both_cells)

In [ ]:
var='flat'
vmin=0
vmax=500
# vmin,vmax=None, None
cmap=blues
map_mesoNH_timestamp_restrict(mesoNH, 
                     var,
                     vmin=vmin, vmax=vmax, cmap=cmap,
                     timestamp='2021-07-20T12:00:00',
                     add_liaise=True,
                    lon_min=0.6, 
                    lon_max=1.4, 
                    lat_min=41.37, 
                    lat_max=41.92,
                    poly=both_cells
                     )

In [ ]:
#list of all variables names in mesoNH
# mesoNH.data_vars.keys()
varlist=['HBLTOP', 'sens', 'flat', 'rh2m', 't2m',  'LWD', 'LWU', 'SWdnSFC',
          'SWI1_ISBA', 'SWI3_ISBA', 'TG1_ISBA', 'TG3_ISBA']

In [ ]:
for var in varlist:
    vmin=0
    vmax=500
    vmin,vmax=None, None
    cmap=blues
    map_mesoNH_timestamp_restrict(mesoNH, 
                        var,
                        vmin=vmin, vmax=vmax, cmap=cmap,
                        timestamp='2021-07-20T12:00:00',
                        add_liaise=True,
                        # lon_min=0.6, 
                        # lon_max=1.4, 
                        # lat_min=41.37, 
                        # lat_max=41.92,
                        # poly=both_cells
                        )

In [ ]:
var='flat'
vmin=-0
vmax=200
# vmin,vmax=None, None
cmap=blues
map_mesoNH_mean_restrict(mesoNH, 
                 var,
                 vmin=vmin, vmax=vmax, cmap=cmap,
                 title='ISBA latent heat flux mean (14-31 July)',
                 add_liaise=True,
                lon_min=0.6, 
                lon_max=1.4, 
                lat_min=41.37, 
                lat_max=41.92,
                poly=both_cells,
                label='Latent heat flux (W m⁻²)',
                 )

In [ ]:
map_mesoNH_timestamp(mesoNH_elsplans_cell,
                 var='flat',
                 vmin=0, vmax=250, cmap=myvir,
                #  title='Latent Heat Flux mean (14-31/07/2021) in Ebro basin',
                 add_liaise=True,
                 timestamp='2021-07-20T12:00:00',
                 )

## Means and bins

In [ ]:
ds=mesoNH_cendrosa_cell
# ds=mesoNH_elsplans_cell
timestamp='2021-07-20T12:00:00'
var='sens'
xmin=0
xmax=550
nbins=11
bins_timestamp(ds, var, timestamp, nbins=nbins, xmin=xmin, xmax=xmax)

In [ ]:
ds=mesoNH_cendrosa_cell
ds=mesoNH_elsplans_cell
# timestamp='2021-07-20T12:00:00'
var='flat'
xmin=0
xmax=525
nbins=21
title='Distribution of mean latent heat flux at 12UTC (14-31/07/2021)'
xlabel='Latent heat flux (W m⁻²)'
bins_hour(ds, var, 12, nbins=nbins, xmin=xmin, xmax=xmax, 
          title=title, xlabel=xlabel)

In [ ]:
vars=['flat', 'sens']
ds1=cendrosa_exact
ds2=mesoNH_cendrosa_cell
ds3=cen_lmdz_noirr
ds4=cen_lmdz_irr100
ds5=cen_lmdz_routingfull_beta1
ds6=cen_mto

ds8=elsplans_exact
ds9=mesoNH_elsplans_cell
ds10=els_lmdz_noirr
ds11=els_lmdz_irr100
ds12=els_lmdz_routingfull_beta1
ds13=els_mto

for var in vars:
    for ds in [ds1, ds2, ds3, ds4, ds5, ds6, ds8, ds9, ds10, ds11, ds12, ds13]:
            mean = mean_value_hour(ds, var, 12)
            print(f"{ds.name} - {var} at 12 UTC: {mean:.2f} W m⁻²")

## Time series

In [ ]:
var='t2m'
ds=mesoNH
vmin=0
vmax=200
vmin,vmax=None, None
time_series_lonlat_mesoNH(ds, var, 
                          lon=ElsPlans_lon, 
                          lat=ElsPlans_lat,
                          vmin=vmin, vmax=vmax
                          )

In [ ]:
var='sens'
ds=mesoNH_cendrosa_cell
ds=mesoNH_elsplans_cell
vmin=0
vmax=200
vmin,vmax=None, None
time_series_ave_mesoNH(ds, var, 
                    #    vmin=vmin, vmax=vmax
                       )

### TS and DC

In [ ]:
var='q2m'
ds1= cen_lmdz_noirr
ds2= cen_lmdz_irr100
ds3= cen_lmdz_routingfull_beta1
ds4=mesoNH_cendrosa_cell
ds5=cendrosa_exact
ds6=restrict_cen_mto

ds1= els_lmdz_noirr
ds2= els_lmdz_irr100
ds3= els_lmdz_routingfull_beta1
ds4=mesoNH_elsplans_cell
ds5=elsplans_exact
ds6=restrict_els_mto

ds4.attrs['show_envelope'] = True

# ds1['q2m'].attrs['long_name'] = '2-m specific humidity'

ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]  
# ds_list=[ds6, ds1, ds2, ds3]  
ds_list=[ds6]

# title=f'{ds1[var].attrs['long_name']} ({ds1[var].attrs['units']})'
# if ds1[var].attrs['units'] == 'W m⁻²':
#     title=f'{ds1[var].attrs["long_name"]} (W m⁻²)'
title='off'

# ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
# ylabel = ds1[var].attrs['name'] + ' (' + ds1[var].attrs['units'] + ')'
ylabel=var + ' (' + ds1[var].attrs['units'] + ')'

In [ ]:
vmin=-0
vmax=100
vmin,vmax=None, None

time_series_ave(ds_list, var, 
                vmin=vmin, vmax=vmax,
                title=title,
                legend_out=False,
                xlabel='Date',
                ylabel=ylabel,
                ds_colors=True,
                figsize=(7, 4),
                envelope=False
                )

In [ ]:
vmin=0
vmax=100
vmin,vmax=None, None

diurnal_cycle_ave(ds_list, var,
                 vmin=vmin, vmax=vmax,
                 title=title,
                 legend_out=None,
                 xlabel='Time (UTC)',
                 ylabel=ylabel,
                 ds_colors=True,
                figsize=(4,4),
                envelope=True
                 )

# Figures

## Surface variables on both sites, time series and mean diurnal cycle (15-31/07)

### La Cendrosa

In [ ]:
savefig=True
plot_TS=True
plot_DC=True

varlist=['flat', 'sens', 't2m']
varlist=['q2m']

ds1= cen_lmdz_noirr
ds2= cen_lmdz_irr100
ds3= cen_lmdz_routingfull_beta1
ds4=mesoNH_cendrosa_cell
ds4.attrs['show_envelope'] = True
ds5=cendrosa_exact
ds6=restrict_cen_mto
ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]  

title='off'

#TS
mins=[0,-150,15,0]
maxs=[700,450,35,0.015]
for var in varlist:
    if not plot_TS:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False
                    )
    if savefig:
        plt.savefig(f'figures/chap5/time_series_cendrosa_{var}.png', dpi=300)

#DC
mins=[0,-150,15,0]
maxs=[520,420,35,0.015]
for var in varlist:
    if not plot_DC:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    diurnal_cycle_ave(ds_list, var,
                     vmin=vmin, vmax=vmax,
                     title=title,
                     legend_out=True,
                    #  xlabel='Time (UTC)',
                     ylabel=ylabel,
                     ds_colors=True,
                    figsize=(4,4),
                    envelope=True
                     )
    if savefig:
        plt.savefig(f'figures/chap5/diurnal_cycle_cendrosa_{var}.png', dpi=300, bbox_inches='tight')

In [ ]:
#no mesoNH
savefig=True
plot_TS=False
plot_DC=True

varlist=['wind_speed_10m', 'wind_direction_10m']

ds1= cen_lmdz_noirr
ds2= cen_lmdz_irr100
ds3= cen_lmdz_routingfull_beta1
ds6=restrict_cen_mto
ds_list= [ds6, ds1, ds2, ds3]  

title='off'

#TS
for var in varlist:
    if not plot_TS:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False
                    )
    if savefig:
        plt.savefig(f'figures/chap5/time_series_cendrosa_{var}.png', dpi=300)

#DC
for var in varlist:
    if not plot_DC:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    diurnal_cycle_ave(ds_list, var,
                     vmin=vmin, vmax=vmax,
                     title=title,
                     legend_out=True,
                    #  xlabel='Time (UTC)',
                     ylabel=ylabel,
                     ds_colors=True,
                    figsize=(4,4),
                    envelope=True
                     )
    if savefig:
        plt.savefig(f'figures/chap5/diurnal_cycle_cendrosa_{var}.png', dpi=300  , bbox_inches='tight')

### Els Plans

In [ ]:
savefig=True
plot_TS=True
plot_DC=True

varlist=['flat', 'sens', 't2m', 'rh2m']
varlist=['q2m']

ds1= els_lmdz_noirr
ds2= els_lmdz_irr100
ds3= els_lmdz_routingfull_beta1
ds4=mesoNH_elsplans_cell
ds4.attrs['show_envelope'] = True
ds5=elsplans_exact
ds6=restrict_els_mto
ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]

title='off'
#TS
mins=[0,-150,15,0]
maxs=[700,450,35,0.015]
for var in varlist:
    if not plot_TS:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False
                    )
    if savefig:
        plt.savefig(f'figures/chap5/time_series_elsplans_{var}.png', dpi=300)
#DC
mins=[0,-150,15,0]
maxs=[520,420,35,0.015]
for var in varlist:
    if not plot_DC:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    diurnal_cycle_ave(ds_list, var,
                     vmin=vmin, vmax=vmax,
                     title=title,
                     legend_out=True,
                    #  xlabel='Time (UTC)',
                     ylabel=ylabel,
                     ds_colors=True,
                    figsize=(4,4),
                    envelope=True
                     )
    if savefig:
        plt.savefig(f'figures/chap5/diurnal_cycle_elsplans_{var}.png', dpi=300, bbox_inches='tight')